<a href="https://colab.research.google.com/github/spirosChv/smartNetsWorkshop/blob/main/neuron/practical2_basicNeuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The NEURON Simulator

NEURON is a simulator for neurons and networks of neurons that runs efficiently on your local machine, in the cloud, or on an HPC. Build and simulate models using Python, HOC, and/or NEURON’s graphical interface.

In [1]:
!pip install neuron --quiet

     |████████████████████████████████| 15.0 MB 39.7 MB/s 


## Step 1: Import the neuron module into Python.

We begin by loading NEURON’s h module.

In [2]:
from neuron import h
h.load_file('stdrun.hoc')

## Step 2: Create a cell

We create a simple model cell as a NEURON `Section`. Evaluate the line below:

In [ ]:
soma = h.Section(name='soma')

### Aside 1: NEURON’s psection() function.

NEURON’s `psection()` (short for “print section”) function can provide a lot of detail on sections. Let’s validate that we have a soma and view some of its properties.

In [ ]:
h.psection()

The results tell us the soma is a cylinder with length $100 microns$, diameter $500 microns$, axial resistivity $35.4 ohm*cm$, and specific membrance capacitance $1 \mu F/cm^2$

### Aside 2: Python’s dir() function.

We can also probe objects with Python’s built-in `dir()` function. Let’s see what it says about soma.

In [ ]:
dir(soma)

This tells us all of the Python methods and variables associated with the object. Any methods with two leading and trailing underscores are reserved by Python and may or may not be implemented by the object. The other items in the list are additional members of soma that we can call. To see all of the functions available to the neuron variable h, try calling `dir(h)`.

### Aside 3: Getting more help.

In addition to probing objects with `dir()`, help from docstrings is available using `help()`.

```python
help(object_or_variable)
```

For example, from `dir(soma)`, we know that `connect` is available. Let’s inquire about that.

In [2]:
help(soma.connect)

## Step 3: Biophysical Mechanisms

NEURON comes with a few built in biophysical mechanisms that can be added to a model.

- `pas`: Passive (“leak”) channel.
- `extracellular`: For simulating effects of nonzero extracellular potential, as may happen with leaky patch clamps, or detailed propertes of the myelin sheath.
- `hh`:	Hodgkin-Huxley sodium, potassium, and leakage channels.

### Aside 3: Insert a passive mechanism.

We see from the list of elements after calling `dir(soma)` that `insert` is available. This is the method we will use to insert mechanisms into the membrane. Let’s insert a passive leak conductance across the membrane and do this by passing `pas` as the mechanism type:

In [2]:
soma.insert('pas')

### IMPORTANT: Sections and segments.

A NEURON Section is considered a piece of cylindrical cable. Depending on the resolution desired, it may be necessary to divide the cable into a number of segments where voltage varies linearly between centers of adjacent segments. The number of segments within a section is given by the variable, `nseg`. The total ionic current across the segment membrane is approximately the area of the segment multiplied by the ionic current density at the center of the segment. To access a part of the section, specify a value between 0 and 1, where 0 is typically the end closest to the soma and 1 is the distal end. Because nseg divides the cable into equal-length parts, it should be an odd number so that to address the middle of the cable, (0.5), gives the middle segment.

To summarize, we access sections by their name and segments by some location on the section.

Section: `section`
Segment: `section(loc)`


Using the Python `type()` function can tell us what a variable is:

In [ ]:
print("type(soma) = {}".format(type(soma)))
print("type(soma(0.5)) ={}".format(type(soma(0.5))))

### Aside 4: Accessing segment variables.

Segment variables follow the idiom:

```python
section(loc).var
```

And for mechanisms on the segment:

```python
section(loc).mech.var
```

or

```python
section(loc).var_mech
```

The first form is preferred.

In [ ]:
mech = soma(0.5).pas
print(dir(mech))

In [ ]:
print(mech.g)
print(soma(0.5).pas.g)

## Step 4: Insert an alpha synapse.

Let’s insert an AlphaSynapse object onto the soma to induce some membrane dynamics.

In [ ]:
asyn = h.AlphaSynapse(soma(0.5))

AlphaSynapse is a **Point Process**. Point processes are point sources of current. When making a new PointProcess, you pass the segment to which it will bind.

Again, with `dir()` function, we can validate that asyn is an object and contains some useful parameters. Let’s look at some of those parameters.

In [ ]:
dir(asyn)

In [ ]:
print("asyn.e = {}".format(asyn.e))
print("asyn.gmax = {}".format(asyn.gmax))
print("asyn.onset = {}".format(asyn.onset))
print("asyn.tau = {}".format(asyn.tau))

Let’s assign the onset of this synapse to occur at 20 ms and the maximal conductance to 1.

In [ ]:
asyn.onset = 20
asyn.gmax = 1

Let’s look at the state of our cell using neuron’s `psection()`.

In [ ]:
h.psection()

## Step 5: Set up recording variables.

The cell should be configured to run a simulation. However, we need to set up variables we wish to record from the simulation. For now, we will record the membrane potential, which is `soma(0.5).v`. References to variables are available as `_ref_rangevariable`.

In [ ]:
v_vec = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

## Step 6: Run the simulation.

To run the simulation, we execute the following lines.

In [ ]:
h.tstop = 40.0
h.run()

**Note:** If we had not included gui in the list of things to import, we would have also had to execute:

`h.load_file('stdrun.hoc')`

which defines the `run()` function (the alternative would be to specify initialization and advance in more detail).

## Step 7: Plot the results.

We utilize the pyplot module from the matplotlib Python package to visualize the output.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
plt.plot(t_vec, v_vec)
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()

The last line displays the graph and allows you to interact with it (zoom, pan, save, etc). NEURON and Python will wait until you close the figure window to proceed.

## Step 8: Saving and restoring results.

NEURON Vector objects can be pickled. This allows them to be stored to disk and restored later:

Saving:

In [ ]:
# Pickle
import pickle
with open('t_vec.p', 'wb') as t_vec_file:
  pickle.dump(t_vec, t_vec_file)
with open('v_vec.p', 'wb') as v_vec_file:
  pickle.dump(v_vec, v_vec_file)

In [ ]:
# Unpickle
with open('t_vec.p', 'rb') as t_vec_file:
  t_vec = pickle.load(t_vec_file)
with open('v_vec.p', 'rb') as vec_file:
  v_vec = pickle.load(vec_file)

# Confirm
plt.figure(figsize=(8,4)) # Default figsize is (8,6)
plt.plot(t_vec, v_vec)
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()